### Implements WDGRL:
- [Wasserstein Distance Guided Representation Learning, Shen et al. (2017)](https://arxiv.org/abs/1707.01217)

### Model Architecture
![](model_archs/Wasserstein_Distance_Guided_Representation_Learning_for_Domain_Adaptation_model_arch.png)

Image borrowed from [Wasserstein Distance Guided Representation Learning, Shen et al. (2017)](https://arxiv.org/abs/1707.01217)

### Make Necessary Imports

In [1]:
import argparse

import torch
from torch import nn
from torch.autograd import grad
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor
from tqdm import tqdm, trange

import config
from data import MNISTM
from models import Net
from utils import loop_iterable, set_requires_grad, GrayscaleToRgb

If CUDA-enabled GPU isn't found, we run on CPU.

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

`gradient_penalty` function definition

In [3]:
def gradient_penalty(critic, h_s, h_t):
    # based on: https://github.com/caogang/wgan-gp/blob/master/gan_cifar10.py#L116
    alpha = torch.rand(h_s.size(0), 1).to(device)
    differences = h_t - h_s
    interpolates = h_s + (alpha * differences)
    interpolates = torch.stack([interpolates, h_s, h_t]).requires_grad_()

    preds = critic(interpolates)
    gradients = grad(preds, interpolates,
                     grad_outputs=torch.ones_like(preds),
                     retain_graph=True, create_graph=True)[0]
    gradient_norm = gradients.norm(2, dim=1)
    gradient_penalty = ((gradient_norm - 1)**2).mean()
    return gradient_penalty

### Set necessary hyperparameters

In [5]:
MODEL_FILE = 'trained_models/source.pt'
batch_size = 64
iterations = 500
epochs = 5
k_critic = 5
k_clf = 1
gamma = 10
wd_clf = 1

### Train WDGRL model 

In [6]:
clf_model = Net().to(device)
clf_model.load_state_dict(torch.load(MODEL_FILE))

feature_extractor = clf_model.feature_extractor
discriminator = clf_model.classifier

critic = nn.Sequential(
    nn.Linear(320, 50),
    nn.ReLU(),
    nn.Linear(50, 20),
    nn.ReLU(),
    nn.Linear(20, 1)
).to(device)

half_batch = batch_size // 2
source_dataset = MNIST(config.DATA_DIR/'mnist', train=True, download=True,
                      transform=Compose([GrayscaleToRgb(), ToTensor()]))
source_loader = DataLoader(source_dataset, batch_size=half_batch, drop_last=True,
                           shuffle=True, num_workers=0, pin_memory=True)

target_dataset = MNISTM(train=False)
target_loader = DataLoader(target_dataset, batch_size=half_batch, drop_last=True,
                           shuffle=True, num_workers=0, pin_memory=True)

critic_optim = torch.optim.Adam(critic.parameters(), lr=1e-4)
clf_optim = torch.optim.Adam(clf_model.parameters(), lr=1e-4)
clf_criterion = nn.CrossEntropyLoss()

for epoch in range(1, epochs+1):
    batch_iterator = zip(loop_iterable(source_loader), loop_iterable(target_loader))

    total_loss = 0
    total_accuracy = 0
    for _ in trange(iterations, leave=False):
        (source_x, source_y), (target_x, _) = next(batch_iterator)
        # Train critic
        set_requires_grad(feature_extractor, requires_grad=False)
        set_requires_grad(critic, requires_grad=True)

        source_x, target_x = source_x.to(device), target_x.to(device)
        source_y = source_y.to(device)

        with torch.no_grad():
            h_s = feature_extractor(source_x).data.view(source_x.shape[0], -1)
            h_t = feature_extractor(target_x).data.view(target_x.shape[0], -1)
        for _ in range(k_critic):
            gp = gradient_penalty(critic, h_s, h_t)

            critic_s = critic(h_s)
            critic_t = critic(h_t)
            wasserstein_distance = critic_s.mean() - critic_t.mean()

            critic_cost = -wasserstein_distance + gamma*gp

            critic_optim.zero_grad()
            critic_cost.backward()
            critic_optim.step()

            total_loss += critic_cost.item()

        # Train classifier
        set_requires_grad(feature_extractor, requires_grad=True)
        set_requires_grad(critic, requires_grad=False)
        for _ in range(k_clf):
            source_features = feature_extractor(source_x).view(source_x.shape[0], -1)
            target_features = feature_extractor(target_x).view(target_x.shape[0], -1)

            source_preds = discriminator(source_features)
            clf_loss = clf_criterion(source_preds, source_y)
            wasserstein_distance = critic(source_features).mean() - critic(target_features).mean()

            loss = clf_loss + wd_clf * wasserstein_distance
            clf_optim.zero_grad()
            loss.backward()
            clf_optim.step()

    mean_loss = total_loss / (iterations * k_critic)
    tqdm.write(f'EPOCH {epoch:03d}: critic_loss={mean_loss:.4f}')
    torch.save(clf_model.state_dict(), 'trained_models/wdgrl.pt')

EPOCH 001: critic_loss=-6.0905


EPOCH 002: critic_loss=-5.8040


EPOCH 003: critic_loss=-2.1073


EPOCH 004: critic_loss=-1.1040


EPOCH 005: critic_loss=-0.7103
